# WebScrapping - Compilateur d'odfres de stages / emplois

## Imports

In [53]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
%reload_ext autoreload

In [3]:
import bs4
from bs4 import BeautifulSoup
import selenium
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import copy
import streamlit as st
from utils.selenium import element_exists, bypass_captcha, wait_for, hcaptcha_solution
from utils.login import login_google, login_indeed
from utils.threading import init_drivers




# code qui marche

In [81]:
from utils.threading import init_drivers
from utils.processing.linkedin import setup_linkedin
from utils.login import login_glassdoor,login_linkedin
from utils.selenium import create_driver
l_email = "scrapperselenium@gmail.com"
l_password= "Password123!*"

g_email = "severin.lefebure@edu.devinci.fr"
g_password = "jesuisungenie"
job = "Data Analyst"
city = "Paris"
#5 drivers
#do threads
l_drivers = init_drivers(1,func=setup_linkedin, args=[None, l_email, l_password, False, False])
g_drivers = init_drivers(3,func=login_glassdoor, args=[None, g_email, g_password, False, False])
useless_drivers = init_drivers(1, func=create_driver, args=[False])
drivers = l_drivers+g_drivers+useless_drivers
# l_drivers = 0
# g_drivers = 1,2,3
# e_drivers = 4 
print(drivers)

len_driver 1 [[]]
0
range: 1
<ThreadWithReturnValue(Thread-195 (setup_linkedin), initial)>
Creating driver
return None
<selenium.webdriver.chrome.webdriver.WebDriver (session="8e83c75014fe9d3809e119ac55cf43d1")>
len_driver 1 [[]]
0
range: 3
<ThreadWithReturnValue(Thread-198 (login_glassdoor), initial)>
Creating driver
return None
<ThreadWithReturnValue(Thread-199 (login_glassdoor), initial)>
Creating driver
return None
<ThreadWithReturnValue(Thread-200 (login_glassdoor), initial)>
Creating driver
return None
<selenium.webdriver.chrome.webdriver.WebDriver (session="76c8b032fd456a6e2b1c3e1226c4376a")>
<selenium.webdriver.chrome.webdriver.WebDriver (session="b876f3744a3002e2d45e18e37491f723")>
<selenium.webdriver.chrome.webdriver.WebDriver (session="42adffb90b27cd75297b61b3533047ef")>
len_driver 1 [[]]
0
range: 1
<ThreadWithReturnValue(Thread-207 (create_driver), initial)>
return None
<selenium.webdriver.chrome.webdriver.WebDriver (session="57bfccae9231cfa30df71d8437f65460")>
[[<selenium.

In [52]:
from selenium.webdriver.common.action_chains import ActionChains

dico_filter = {}

filter_panel = drivers[0][0].find_element(By.CSS_SELECTOR, '[aria-labelledby="reusable-search-advanced-filters-right-panel"]')
filters = filter_panel.find_elements(By.CLASS_NAME, "search-reusables__secondary-filters-filter")#"search-reusables__filter-value-item")
for filter in filters:
    hover = ActionChains(drivers[0][0]).move_to_element(filter)
    hover.perform()
    print("ok")
    filter_name = filter.find_element(By.TAG_NAME, "h3").text
    print("filter:", filter_name)
    try:
        filter_info = filter.find_element(By.CLASS_NAME, "search-reusables__secondary-filters-values")
        filter_text = filter_info.text.split("\n")[::2]
        filters_tag = filter_info.find_elements(By.TAG_NAME, "input")
        tag_list = []
        for tag in filters_tag:
            tag_id = tag.get_attribute("id")
            print(tag_id, end="|")
            tag_id = tag_id.replace("advanced-filter-","").replace("-","=")
            tag_list.append(tag_id)
            capital_letters = tag_id[0].upper() + "".join([letter for letter in tag_id[1:] if letter.isupper()])
            REAL_FILTER = f"&f_{capital_letters}"
            print(REAL_FILTER)
            
        dico_filter[filter_name] = tag_list
        print()
        #print(filter_text, tag_list)
    except:
        dico_filter[filter_name] = []
        print("no filter")

ok
filter: Classer par
advanced-filter-sortBy-DD|&f_SBDD
advanced-filter-sortBy-R|&f_SBR

ok
filter: Date de publication
advanced-filter-timePostedRange-|&f_TPR
advanced-filter-timePostedRange-r2592000|&f_TPR
advanced-filter-timePostedRange-r604800|&f_TPR
advanced-filter-timePostedRange-r86400|&f_TPR

ok
filter: Niveau d’expérience
advanced-filter-experience-1|&f_E
advanced-filter-experience-2|&f_E
advanced-filter-experience-3|&f_E
advanced-filter-experience-4|&f_E
advanced-filter-experience-5|&f_E
advanced-filter-experience-6|&f_E

ok
filter: Entreprise
advanced-filter-company-11084085|&f_C
advanced-filter-company-1258653|&f_C
advanced-filter-company-12627290|&f_C
advanced-filter-company-93089723|&f_C
advanced-filter-company-2536626|&f_C
advanced-filter-company-9485372|&f_C
advanced-filter-company-66719|&f_C
advanced-filter-company-3896|&f_C
advanced-filter-company-68841835|&f_C
advanced-filter-company-1691|&f_C
advanced-filter-company-498995|&f_C
advanced-filter-company-2046490|&f_C


In [82]:
drivers2 = []
for driverss in drivers:
    for driver in driverss:
        drivers2.append(driver)
drivers2

[<selenium.webdriver.chrome.webdriver.WebDriver (session="8e83c75014fe9d3809e119ac55cf43d1")>,
 <selenium.webdriver.chrome.webdriver.WebDriver (session="76c8b032fd456a6e2b1c3e1226c4376a")>,
 <selenium.webdriver.chrome.webdriver.WebDriver (session="b876f3744a3002e2d45e18e37491f723")>,
 <selenium.webdriver.chrome.webdriver.WebDriver (session="42adffb90b27cd75297b61b3533047ef")>,
 <selenium.webdriver.chrome.webdriver.WebDriver (session="57bfccae9231cfa30df71d8437f65460")>]

In [4]:
for driver_list in drivers2:
    for driver in driver_list:
        driver.quit()

AttributeError: 'NoneType' object has no attribute 'quit'

In [98]:
### Scrap the page data
#drivers = [main_driver, lookup_driver]
from utils.scrapping.linkedin import get_job_data
get_job_data(drivers2, job, city, verbose=True, bypass=True, limit=1)

get job data
4
**Time of pre-loop Bypass :**  0.1037743091583252
New_element
['https:', '', 'www.linkedin.com', 'jobs', 'view', '3795100435']
scrap job info
company: Tata Consultancy Services
get company info
1
scrap company info
scrap reviews_info
----> Time of eco_score 2.6056394577026367
scores
[('Diversité et inclusion', '4,0'), ('Culture et valeurs', '3,9'), ('Équilibre travail/vie privée', '3,8'), ('Opportunités de carrière', '3,7'), ('Équipe dirigeante', '3,3'), ('Rémunération et avantages', '3,1')] [('5 étoiles', '31 %'), ('4 étoiles', '38 %'), ('3 étoiles', '23 %'), ('2 étoiles', '5 %'), ('1 étoile', '3 %')]
-------> time of append : 6.78360652923584
1 24 [['DATA TECH LEAD', '3795100435', 'Tata Consultancy Services', [('job', 'Sur site Temps plein Confirmé'), ('company', '+ de 10 000 employés · Services et conseil en informatique'), ('checklist', 'Compétences : Python (langage de programmation), Scala et 1 en plus'), ('verified', 'Consultez les vérifications liées à cette offr

,Job Title,Job ID,Company Name,Job Tags,Job Loc,Ecoscore,Company name,Company infos,verified,checklist,...,Diversité et inclusion,Rémunération et avantages,Équilibre travail/vie privée,Culture et valeurs,Opportunités de carrière,4 étoiles,3 étoiles,5 étoiles,2 étoiles,1 étoile
0,DATA TECH LEAD,3795100435,Tata Consultancy Services,"[(job, Sur site Temps plein Confirmé), (compan...","{'Ville': 'Ville de Paris', 'Région': 'Île-de-...",0,Tata Consultancy Services,"[www.tcs.com/, Mumbai, Inde, Plus de 10 000 em...",Consultez les vérifications liées à cette offr...,Compétences : Python (langage de programmation...,...,"4,0","3,1","3,8","3,9","3,7",38 %,23 %,31 %,5 %,3 %


In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [6]:
df = pd.read_csv("List_jobs_One.csv")
df
df.to_excel("List_jobs.xlsx")